# Image matching dataset
- Download the dataset from Kaggle competition page: [Image Matching Challenge 2025](https://www.kaggle.com/competitions/image-matching-challenge-2025/data)
- Custom Dataset class to load the dataset
- Normalize the images with mean and std of 0.5 for all channels.

In [8]:
from data_preprocess.image_matching_dataset import ImageMatchingDataset
import torchvision.transforms as transforms
import torch

train_dataset = ImageMatchingDataset(labels_path='data/train_labels.csv', root_dir='data/train',
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
                               ]))

## Initial Analysis of the dataset

In [11]:
from torch.utils.data import DataLoader

DEVICE = torch.device(0 if torch.cuda.is_available() else -1)

'''Create a dataloader for train dataset such that each batch contains images from same dataset class'''
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=1)
